In [ ]:
from reina.IvMethods.TSLS import SparkSieveTSLS
from pyspark.sql import SparkSession

# Initialize spark session
spark = SparkSession \
            .builder \
            .appName('Meta-Learner-Spark') \
            .getOrCreate()

#### Read toy data. Replace .load() with the test_data.csv location -- this location could be a local one (no cluster) or it could be on a distributed storage system (e.g., HDFS)

*Note: Code below assumes data generated by our script (for specifics, please refer to our toy data generation in the README). You could also modify the code accordingly to use your own data.*

In [ ]:
df = spark.read \
          .format("csv") \
          .option('header', 'true') \
          .load("test_data.csv")  # replace with the location of test_data.csv

# Case variables to appropriate types
df = df.withColumn("var1", df.var1.cast("float"))
df = df.withColumn("var2", df.var2.cast("float"))
df = df.withColumn("var3", df.var3.cast("float"))
df = df.withColumn("var4", df.var4.cast("float"))
df = df.withColumn("var5", df.var5.cast("float"))
df = df.withColumn("treatment", df.treatment.cast("float"))
df = df.withColumn("outcome", df.outcome.cast("float"))

# Drop garbage column
df = df.drop("_c0")

# Print out dataframe schema
print(df.schema)

#### Two-stage Least Squares

In [ ]:
# Set up necessary parameters
treatments = ['treatment']
outcome = 'outcome'
iv = 'var1'

# Fit TSLS
spark_tsls = SparkSieveTSLS()
spark_tsls.fit(data=df, treatments=treatments, outcome=outcome, iv=iv)

# Get heterogeneous treatment effects (cate for individual samples and ate for averaged treatment effect)
cate, ate = spark_tsls.effect()
print(cate)  
print(ate)